In [1]:
import tweepy
import pickle
import datetime
import random

In [6]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [3]:
def get_tweets(auth, woeid):
    """
    Save a list of trending tweets based on woeid (i.e. georgraphical location)
    """

    # Authenticate
    api = tweepy.API(auth, wait_on_rate_limit=True,
                     wait_on_rate_limit_notify=True, compression=True)

    # Get list of trending topics
    list_of_trends = api.trends_place(woeid)[0]['trends']
    # list_of_canada_trends = api.trends_place(23424775)[0]['trends']
    # print(list_of_trends)

    # Get about 15 tweets from each trend (50 trends)
    # About 750 tweets
    list_of_tweets = []

    for trend in list_of_trends:
        trend_query = trend['query']
        trending_tweets = api.search(trend_query, lang='en')

        for tweet in trending_tweets:
            list_of_tweets.append(tweet)

    # Save for use later
    pickle.dump(list_of_tweets, open("tweets_NZ.p", "wb"))


In [4]:
def get_friend_tweets(tweets, bound, tweet_required, following_limit):
    """
    Save required information required for analysis.  tweets is a list of tweet, bound is the time requirement (e.g. one hour or 2 hour before tweet).
    tweet_required is the approximate limit for tweets gather from each friend and following_limit is the the limit on the follower that we go through
    for each friend of the original tweet
    """

    # Get all tweets generated in the last hour (hours specified by bound)
    tweets_related = []
    time_limit = datetime.timedelta(hours=bound)
    index = 1

    for tweet in tweets:
        # Information to be stored in a list - {User ID, Tweet ID, Time, Following, Tweets, Other}
        print("Going through tweet " + str(index) + " of  " + str(len(tweets)))
        index += 1
        tweet_info = {}

        # Tweets and info related to the friends of the author of tweet
        important_tweet = []
        others = []

        # Tweet info
        user_id = tweet.user.id
        tweet_id = tweet.id
        time_created = tweet.created_at
        followings = api.friends_ids(user_id)

        # Lower bound for inclusion in important tweet
        time_lower_bound = time_created - time_limit

        # Go through all the followers, get tweet_required of their tweets, if
        # it's posted within an hour of tweet, append it to important tweet
        f = 1
        for following in followings[:following_limit]:
            print("Going through friends " + str(f) + " of " + str(len(followings[:following_limit])))
            f += 1
            try:
                other = api.user_timeline(user_id=following, count=tweet_required, include_rts=False, tweet_mode='extended')
                others.append(other)

                for other_tweet in other:
                    other_tweet_created = other_tweet.created_at

                    if time_created >= other_tweet_created >= time_lower_bound:
                        important_tweet.append(other_tweet)
            except tweepy.TweepError:
                print("error")
                continue

        # Store information generated in dictionary
        tweet_info['user'] = user_id
        tweet_info['id'] = tweet_id
        tweet_info['created'] = time_created
        tweet_info['info'] = tweet
        tweet_info['other'] = others
        tweet_info['related_tweets'] = important_tweet

        tweets_related.append(tweet_info)
        pickle.dump(tweets_related, open("temp.p", "ab"))        

    # Make sure this object is available for future use
    pickle.dump(tweets_related, open("friend_tweets_NZ.p", "wb"))

In [7]:
api = tweepy.API(auth, wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True, compression=True)

# Get trending English tweets in a specific location, 23424916 is New Zealand
get_tweets(auth, 23424916)

# If available, load the tweets generated previously
# tweets.p can be replaced with any file that you have generated
tweets = pickle.load(open("tweets.p", "rb"))

# Get the first 300 tweets first (specified with test_tweet), if have time,
# do more.  Limit search to 200 friends.  For each tweet in tweets, go 
# through all the friend (i.e. who the user is following) of the user, 
# then get the tweet of each of the friend that is posted within 1 hour.
test_tweet = tweets[0:300]
get_friend_tweets(test_tweet, 1, 5, 200)

Going through tweet 1 of  300
Going through friends 1 of 200
Going through friends 2 of 200
Going through friends 3 of 200
Going through friends 4 of 200
Going through friends 5 of 200
Going through friends 6 of 200
Going through friends 7 of 200
Going through friends 8 of 200
Going through friends 9 of 200
Going through friends 10 of 200
Going through friends 11 of 200
Going through friends 12 of 200
Going through friends 13 of 200
Going through friends 14 of 200
Going through friends 15 of 200
Going through friends 16 of 200
Going through friends 17 of 200
Going through friends 18 of 200
Going through friends 19 of 200
Going through friends 20 of 200
Going through friends 21 of 200
Going through friends 22 of 200
Going through friends 23 of 200
Going through friends 24 of 200
Going through friends 25 of 200
Going through friends 26 of 200
Going through friends 27 of 200
Going through friends 28 of 200
Going through friends 29 of 200
Going through friends 30 of 200
Going through frien

Going through friends 51 of 200
Going through friends 52 of 200
Going through friends 53 of 200
Going through friends 54 of 200
Going through friends 55 of 200
Going through friends 56 of 200
Going through friends 57 of 200
Going through friends 58 of 200
Going through friends 59 of 200
Going through friends 60 of 200
error
Going through friends 61 of 200
Going through friends 62 of 200
Going through friends 63 of 200
Going through friends 64 of 200
Going through friends 65 of 200
Going through friends 66 of 200
Going through friends 67 of 200
Going through friends 68 of 200
Going through friends 69 of 200
Going through friends 70 of 200
Going through friends 71 of 200
Going through friends 72 of 200
Going through friends 73 of 200
Going through friends 74 of 200
error
Going through friends 75 of 200
Going through friends 76 of 200
Going through friends 77 of 200
Going through friends 78 of 200
Going through friends 79 of 200
Going through friends 80 of 200
Going through friends 81 of 

Going through friends 99 of 200
Going through friends 100 of 200
Going through friends 101 of 200
error
Going through friends 102 of 200
Going through friends 103 of 200
Going through friends 104 of 200
Going through friends 105 of 200
Going through friends 106 of 200
Going through friends 107 of 200
Going through friends 108 of 200
Going through friends 109 of 200
Going through friends 110 of 200
Going through friends 111 of 200
Going through friends 112 of 200
Going through friends 113 of 200
Going through friends 114 of 200
Going through friends 115 of 200
Going through friends 116 of 200
Going through friends 117 of 200
Going through friends 118 of 200
Going through friends 119 of 200
Going through friends 120 of 200
Going through friends 121 of 200
Going through friends 122 of 200
Going through friends 123 of 200
Going through friends 124 of 200
Going through friends 125 of 200
Going through friends 126 of 200
Going through friends 127 of 200
Going through friends 128 of 200
Going

Going through friends 150 of 200
Going through friends 151 of 200
Going through friends 152 of 200
Going through friends 153 of 200
Going through friends 154 of 200
Going through friends 155 of 200
Going through friends 156 of 200
Going through friends 157 of 200
Going through friends 158 of 200
Going through friends 159 of 200
Going through friends 160 of 200
Going through friends 161 of 200
Going through friends 162 of 200
Going through friends 163 of 200
Going through friends 164 of 200
error
Going through friends 165 of 200
Going through friends 166 of 200
Going through friends 167 of 200
Going through friends 168 of 200
Going through friends 169 of 200
Going through friends 170 of 200
Going through friends 171 of 200
Going through friends 172 of 200
Going through friends 173 of 200
Going through friends 174 of 200
Going through friends 175 of 200
Going through friends 176 of 200
Going through friends 177 of 200
Going through friends 178 of 200
Going through friends 179 of 200
Goin

Rate limit reached. Sleeping for: 700


...

Going through friends 100 of 200
Going through friends 101 of 200
Going through friends 102 of 200
Going through friends 103 of 200
Going through friends 104 of 200
Going through friends 105 of 200
Going through friends 106 of 200
Going through friends 107 of 200
Going through friends 108 of 200
Going through friends 109 of 200
Going through friends 110 of 200
Going through friends 111 of 200
Going through friends 112 of 200
Going through friends 113 of 200
Going through friends 114 of 200
Going through friends 115 of 200
Going through friends 116 of 200
Going through friends 117 of 200
Going through friends 118 of 200
Going through friends 119 of 200
Going through friends 120 of 200
Going through friends 121 of 200
Going through friends 122 of 200
Going through friends 123 of 200
Going through friends 124 of 200
Going through friends 125 of 200
Going through friends 126 of 200
Going through friends 127 of 200
Going through friends 128 of 200
Going through friends 129 of 200
Going thro

Going through friends 151 of 200
Going through friends 152 of 200
Going through friends 153 of 200
Going through friends 154 of 200
Going through friends 155 of 200
Going through friends 156 of 200
Going through friends 157 of 200
Going through friends 158 of 200
Going through friends 159 of 200
Going through friends 160 of 200
Going through friends 161 of 200
Going through friends 162 of 200
Going through friends 163 of 200
Going through friends 164 of 200
Going through friends 165 of 200
Going through friends 166 of 200
Going through friends 167 of 200
Going through friends 168 of 200
Going through friends 169 of 200
Going through friends 170 of 200
Going through friends 171 of 200
Going through friends 172 of 200
Going through friends 173 of 200
Going through friends 174 of 200
Going through friends 175 of 200
Going through friends 176 of 200
Going through friends 177 of 200
Going through friends 178 of 200
Going through friends 179 of 200
Going through friends 180 of 200
Going thro

Rate limit reached. Sleeping for: 588


Going through friends 96 of 200
Going through friends 97 of 200
Going through friends 98 of 200
Going through friends 99 of 200
Going through friends 100 of 200
Going through friends 101 of 200
Going through friends 102 of 200
Going through friends 103 of 200
Going through friends 104 of 200
Going through friends 105 of 200
Going through friends 106 of 200
Going through friends 107 of 200
Going through friends 108 of 200
Going through friends 109 of 200
Going through friends 110 of 200
Going through friends 111 of 200
Going through friends 112 of 200
Going through friends 113 of 200
Going through friends 114 of 200
Going through friends 115 of 200
Going through friends 116 of 200
Going through friends 117 of 200
Going through friends 118 of 200
Going through friends 119 of 200
Going through friends 120 of 200
Going through friends 121 of 200
Going through friends 122 of 200
Going through friends 123 of 200
Going through friends 124 of 200
Going through friends 125 of 200
Going through 

Going through friends 147 of 200
Going through friends 148 of 200
Going through friends 149 of 200
Going through friends 150 of 200
Going through friends 151 of 200
Going through friends 152 of 200
Going through friends 153 of 200
Going through friends 154 of 200
Going through friends 155 of 200
Going through friends 156 of 200
Going through friends 157 of 200
Going through friends 158 of 200
Going through friends 159 of 200
Going through friends 160 of 200
Going through friends 161 of 200
Going through friends 162 of 200
Going through friends 163 of 200
Going through friends 164 of 200
Going through friends 165 of 200
Going through friends 166 of 200
Going through friends 167 of 200
Going through friends 168 of 200
Going through friends 169 of 200
Going through friends 170 of 200
Going through friends 171 of 200
Going through friends 172 of 200
Going through friends 173 of 200
Going through friends 174 of 200
Going through friends 175 of 200
Going through friends 176 of 200
Going thro